In [1]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
from typing import Dict, List, Tuple, Set
from copy import deepcopy

from databases.mongodb import MongoDB
from databases.mongodb_entity import MongoDBEntity
from databases.postgresql import PostgresDB
from src.scripts.tokens_transfers_features import encode_transfers_data_by_coins, encode_transfers_data_by_coins_types

Get token transfers data

In [2]:
chain_id = '0x38'
mongodb = MongoDB(chain_id=chain_id)
mongo_klg = MongoDBEntity()
postgres = PostgresDB()

wallets_pairs = pd.read_csv(f'../../data/{chain_id}_wallets_pairs.csv')

token_transfers_df = pd.read_csv(f'../../data/{chain_id}_tokens_transfers.csv')
print("Number of transfers: ", len(token_transfers_df))
token_transfers_df.head()

Number of transfers:  729514


,contract_address,evt_tx_hash,evt_index,evt_block_time,evt_block_number,from,to,value
0,0x55d398326f99059ff775485246999027b3197955,0xe0f92f3c31d9447d215a1b6dfd531a2cc5b656ac15c8...,1490,2023-02-12 13:52:51.000 UTC,25604754.0,0xc868578486126191e956e9b96f35f691c30a3b5c,0x23d677e9d4e2fb93b7fdcfba230347c1f4bf0580,0
1,0x55d398326f99059ff775485246999027b3197955,0xe0f92f3c31d9447d215a1b6dfd531a2cc5b656ac15c8...,1492,2023-02-12 13:52:51.000 UTC,25604754.0,0xe894755e2f09feef11d9217449be7b00146a503c,0x7430ffc4de6c4372061bf9fd065bd854c3c4038b,0
2,0x55d398326f99059ff775485246999027b3197955,0xe0f92f3c31d9447d215a1b6dfd531a2cc5b656ac15c8...,1494,2023-02-12 13:52:51.000 UTC,25604754.0,0xdbf134b170239da526ac902ca89daafb517baa6d,0x4925bf3d39c46f1da05706e86580cbc464ff72c2,0
3,0x55d398326f99059ff775485246999027b3197955,0xe0f92f3c31d9447d215a1b6dfd531a2cc5b656ac15c8...,1496,2023-02-12 13:52:51.000 UTC,25604754.0,0xd33b58fe34708bf9bfd9adb279ec1edd1a9ff022,0x1557575c8a0073485864575618739431b0b6b1f5,0
4,0x55d398326f99059ff775485246999027b3197955,0xe0f92f3c31d9447d215a1b6dfd531a2cc5b656ac15c8...,1498,2023-02-12 13:52:51.000 UTC,25604754.0,0xc868578486126191e956e9b96f35f691c30a3b5c,0x297c006e42ca8745481520a2ef9c738dd9e506e6,0


Get stable coins & meme coins data

In [3]:
# Stable coins
# symbols:
_stable_coins_cursor = mongo_klg.get_stable_coins(chain_id=chain_id,
                                                  projection={'idCoingecko': 1, 'address': 1, 'symbol': 1})
stable_coins_symbols = {doc['address']: doc['symbol'] for doc in _stable_coins_cursor}
# decimals:
_stablecoins_decimals = postgres.get_decimals(chain_id=chain_id, token_addresses=list(stable_coins_symbols.keys()))
stable_coins_decimals = {row[0]: row[1] for row in _stablecoins_decimals}

# Meme coins
# symbols
_meme_coins_cursor = mongo_klg.get_meme_coins(chain_id=chain_id,
                                              projection={'idCoingecko': 1, 'address': 1, 'symbol': 1})
meme_coins_symbols = {doc['address']: doc['symbol'] for doc in _meme_coins_cursor}
# decimals
_memecoins_decimals = postgres.get_decimals(chain_id=chain_id, token_addresses=list(meme_coins_symbols.keys()))
meme_coins_decimals = {row[0]: row[1] for row in _memecoins_decimals}

print("Number of Stable coins traded: ", len(stable_coins_symbols))
print("Number of Meme coins traded: ", len(meme_coins_symbols))

Number of Stable coins traded:  29
Number of Meme coins traded:  104


## For X wallets

In [4]:
# wallets
wallet_addresses = set(wallets_pairs['x'])
print("Number of wallets: ", len(wallet_addresses))

Number of wallets:  23779


Next, create data frame for 3 features:
- Number of Transfers | Stablecoins transfers | Memecoins transfers
- Stablecoins list transfers
- Memecoins list transfers

In [5]:
stablecoins_sent, n_stablecoins_sent, stablecoins_received, n_stablecoins_received = encode_transfers_data_by_coins(tokens_transfers=token_transfers_df,
                                                                                                                    wallet_addresses=wallet_addresses,
                                                                                                                    coins_symbols=stable_coins_symbols,
                                                                                                                    coins_decimals=stable_coins_decimals)

Progress: 0 / 729514
Progress: 10000 / 729514
Progress: 20000 / 729514
Progress: 30000 / 729514
Progress: 40000 / 729514
Progress: 50000 / 729514
Progress: 60000 / 729514
Progress: 70000 / 729514
Progress: 80000 / 729514
Progress: 90000 / 729514
Progress: 100000 / 729514
Progress: 110000 / 729514
Progress: 120000 / 729514
Progress: 130000 / 729514
Progress: 140000 / 729514
Progress: 150000 / 729514
Progress: 160000 / 729514
Progress: 170000 / 729514
Progress: 180000 / 729514
Progress: 190000 / 729514
Progress: 200000 / 729514
Progress: 210000 / 729514
Progress: 220000 / 729514
Progress: 230000 / 729514
Progress: 240000 / 729514
Progress: 250000 / 729514
Progress: 260000 / 729514
Progress: 270000 / 729514
Progress: 280000 / 729514
Progress: 290000 / 729514
Progress: 300000 / 729514
Progress: 310000 / 729514
Progress: 320000 / 729514
Progress: 330000 / 729514
Progress: 340000 / 729514
Progress: 350000 / 729514
Progress: 360000 / 729514
Progress: 370000 / 729514
Progress: 380000 / 729514


In [6]:
memecoins_sent, n_memecoins_sent, memecoins_received, n_memecoins_received = encode_transfers_data_by_coins(tokens_transfers=token_transfers_df,
                                                                                                            wallet_addresses=wallet_addresses,
                                                                                                            coins_symbols=meme_coins_symbols,
                                                                                                            coins_decimals=meme_coins_decimals)

Progress: 0 / 729514
Progress: 10000 / 729514
Progress: 20000 / 729514
Progress: 30000 / 729514
Progress: 40000 / 729514
Progress: 50000 / 729514
Progress: 60000 / 729514
Progress: 70000 / 729514
Progress: 80000 / 729514
Progress: 90000 / 729514
Progress: 100000 / 729514
Progress: 110000 / 729514
Progress: 120000 / 729514
Progress: 130000 / 729514
Progress: 140000 / 729514
Progress: 150000 / 729514
Progress: 160000 / 729514
Progress: 170000 / 729514
Progress: 180000 / 729514
Progress: 190000 / 729514
Progress: 200000 / 729514
Progress: 210000 / 729514
Progress: 220000 / 729514
Progress: 230000 / 729514
Progress: 240000 / 729514
Progress: 250000 / 729514
Progress: 260000 / 729514
Progress: 270000 / 729514
Progress: 280000 / 729514
Progress: 290000 / 729514
Progress: 300000 / 729514
Progress: 310000 / 729514
Progress: 320000 / 729514
Progress: 330000 / 729514
Progress: 340000 / 729514
Progress: 350000 / 729514
Progress: 360000 / 729514
Progress: 370000 / 729514
Progress: 380000 / 729514


In [7]:
transfers_sent, transfers_received = encode_transfers_data_by_coins_types(tokens_transfers=token_transfers_df,
                                                                          wallet_addresses=wallet_addresses,
                                                                          stable_coin_addresses=set(stable_coins_symbols.keys()),
                                                                          meme_coin_addresses=set(meme_coins_symbols.keys()))

Progress: 0 / 729514
Progress: 10000 / 729514
Progress: 20000 / 729514
Progress: 30000 / 729514
Progress: 40000 / 729514
Progress: 50000 / 729514
Progress: 60000 / 729514
Progress: 70000 / 729514
Progress: 80000 / 729514
Progress: 90000 / 729514
Progress: 100000 / 729514
Progress: 110000 / 729514
Progress: 120000 / 729514
Progress: 130000 / 729514
Progress: 140000 / 729514
Progress: 150000 / 729514
Progress: 160000 / 729514
Progress: 170000 / 729514
Progress: 180000 / 729514
Progress: 190000 / 729514
Progress: 200000 / 729514
Progress: 210000 / 729514
Progress: 220000 / 729514
Progress: 230000 / 729514
Progress: 240000 / 729514
Progress: 250000 / 729514
Progress: 260000 / 729514
Progress: 270000 / 729514
Progress: 280000 / 729514
Progress: 290000 / 729514
Progress: 300000 / 729514
Progress: 310000 / 729514
Progress: 320000 / 729514
Progress: 330000 / 729514
Progress: 340000 / 729514
Progress: 350000 / 729514
Progress: 360000 / 729514
Progress: 370000 / 729514
Progress: 380000 / 729514


In [8]:
stablecoins_sent.to_csv('../../data/transfers_stablecoins_sent_0x38_x.csv')
n_stablecoins_sent.to_csv('../../data/transfers_number_stablecoins_sent_0x38_x.csv')
stablecoins_received.to_csv('../../data/transfers_stablecoins_received_0x38_x.csv')
n_stablecoins_received.to_csv('../../data/transfers_number_stablecoins_received_0x38_x.csv')

memecoins_sent.to_csv('../../data/transfers_memecoins_sent_0x38_x.csv')
n_memecoins_sent.to_csv('../../data/transfers_number_memecoins_sent_0x38_x.csv')
memecoins_received.to_csv('../../data/transfers_memecoins_received_0x38_x.csv')
n_memecoins_received.to_csv('../../data/transfers_number_memecoins_received_0x38_x.csv')

transfers_sent.to_csv('../../data/transfers_number_sent_0x38_x.csv')
transfers_received.to_csv('../../data/transfers_number_received_0x38_x.csv')


## For Y wallets

In [9]:
wallet_addresses = set(wallets_pairs['y'])
print("Number of wallets: ", len(wallet_addresses))

Number of wallets:  23779


In [10]:
# stable coins transfers
stablecoins_sent, n_stablecoins_sent, stablecoins_received, n_stablecoins_received = encode_transfers_data_by_coins(tokens_transfers=token_transfers_df,
                                                                                                                    wallet_addresses=wallet_addresses,
                                                                                                                    coins_symbols=stable_coins_symbols,
                                                                                                                    coins_decimals=stable_coins_decimals)

# meme coins transfers
memecoins_sent, n_memecoins_sent, memecoins_received, n_memecoins_received = encode_transfers_data_by_coins(tokens_transfers=token_transfers_df,
                                                                                                            wallet_addresses=wallet_addresses,
                                                                                                            coins_symbols=meme_coins_symbols,
                                                                                                            coins_decimals=meme_coins_decimals)

# other data transfers
transfers_sent, transfers_received = encode_transfers_data_by_coins_types(tokens_transfers=token_transfers_df,
                                                                          wallet_addresses=wallet_addresses,
                                                                          stable_coin_addresses=set(stable_coins_symbols.keys()),
                                                                          meme_coin_addresses=set(meme_coins_symbols.keys()))

Progress: 0 / 729514
Progress: 10000 / 729514
Progress: 20000 / 729514
Progress: 30000 / 729514
Progress: 40000 / 729514
Progress: 50000 / 729514
Progress: 60000 / 729514
Progress: 70000 / 729514
Progress: 80000 / 729514
Progress: 90000 / 729514
Progress: 100000 / 729514
Progress: 110000 / 729514
Progress: 120000 / 729514
Progress: 130000 / 729514
Progress: 140000 / 729514
Progress: 150000 / 729514
Progress: 160000 / 729514
Progress: 170000 / 729514
Progress: 180000 / 729514
Progress: 190000 / 729514
Progress: 200000 / 729514
Progress: 210000 / 729514
Progress: 220000 / 729514
Progress: 230000 / 729514
Progress: 240000 / 729514
Progress: 250000 / 729514
Progress: 260000 / 729514
Progress: 270000 / 729514
Progress: 280000 / 729514
Progress: 290000 / 729514
Progress: 300000 / 729514
Progress: 310000 / 729514
Progress: 320000 / 729514
Progress: 330000 / 729514
Progress: 340000 / 729514
Progress: 350000 / 729514
Progress: 360000 / 729514
Progress: 370000 / 729514
Progress: 380000 / 729514


In [11]:
stablecoins_sent.to_csv('../../data/transfers_stablecoins_sent_0x38_y.csv')
n_stablecoins_sent.to_csv('../../data/transfers_number_stablecoins_sent_0x38_y.csv')
stablecoins_received.to_csv('../../data/transfers_stablecoins_received_0x38_y.csv')
n_stablecoins_received.to_csv('../../data/transfers_number_stablecoins_received_0x38_y.csv')

memecoins_sent.to_csv('../../data/transfers_memecoins_sent_0x38_y.csv')
n_memecoins_sent.to_csv('../../data/transfers_number_memecoins_sent_0x38_y.csv')
memecoins_received.to_csv('../../data/transfers_memecoins_received_0x38_y.csv')
n_memecoins_received.to_csv('../../data/transfers_number_memecoins_received_0x38_y.csv')

transfers_sent.to_csv('../../data/transfers_number_sent_0x38_y.csv')
transfers_received.to_csv('../../data/transfers_number_received_0x38_y.csv')